In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
import datetime
from io import StringIO
import random
#!{sys.executable} -m pip install xgboost
rs = 420
random.seed(rs)

print(datetime.datetime.today().strftime("%HH:%MM %dd"))
# Load datasets

featuers = ['vesselId', 'hour', 'day', 'month', 'minute', 'vesselType', 'yearBuilt', 'length', 'breadth', 'CEU', 'DWT', 'GT', 'portLatitude', 'portLongitude', 'hour_sch', 'day_sch', 'month_sch', 'minute_sch', 'lat_shift_1', 'lon_shift_1', 'minute_shift_1', 'hour_shift_1', 'day_shift_1', 'month_shift_1', 'cog_1', 'heading_1', 'lat_shift_2', 'lon_shift_2', 'minute_shift_2', 'hour_shift_2', 'day_shift_2', 'month_shift_2', 'cog_2', 'heading_2', 'lat_shift_3', 'lon_shift_3', 'minute_shift_3', 'hour_shift_3', 'day_shift_3', 'month_shift_3', 'cog_3', 'heading_3', 'lat_shift_4', 'lon_shift_4', 'minute_shift_4', 'hour_shift_4', 'day_shift_4', 'month_shift_4', 'cog_4', 'heading_4', 'lat_shift_5', 'lon_shift_5', 'minute_shift_5', 'hour_shift_5', 'day_shift_5', 'month_shift_5', 'cog_5', 'heading_5', 'cog', 'heading', 'time_diff_1', 'time_diff_2', 'time_diff_3', 'time_diff_4', 'time_diff_5']
featuers2 = ['longitude', 'latitude', 'vesselId', 'hour', 'day', 'month', 'minute', 'vesselType', 'length', 'breadth', 'CEU', 'DWT', 'GT', 'portLatitude', 'portLongitude', 'lat_shift_1', 'lon_shift_1', 'lat_shift_2', 'lon_shift_2', 'lat_shift_3', 'lon_shift_3', 'lat_shift_4', 'lon_shift_4', 'lat_shift_5', 'lon_shift_5', 'time_diff_1', 'time_diff_2', 'time_diff_3', 'time_diff_4', 'time_diff_5']
target2 = ['cog', 'heading']
vessel_encoder = LabelEncoder()
timesteps = 6
# test-test split
liste = []
for i in range(len(featuers)):
    liste.append(pd.read_csv('x_data_'+str(i)+'.csv', sep=','))

featuers = ['vesselId', 'hour', 'day', 'month', 'minute', 'vesselType', 'length', 'breadth', 'CEU', 'DWT', 'GT', 'portLatitude', 'portLongitude', 'lat_shift_1', 'lon_shift_1', 'lat_shift_2', 'lon_shift_2', 'lat_shift_3', 'lon_shift_3', 'lat_shift_4', 'lon_shift_4', 'lat_shift_5', 'lon_shift_5', 'cog', 'heading', 'time_diff_1', 'time_diff_2', 'time_diff_3', 'time_diff_4', 'time_diff_5']
X = liste[0].join(liste[1:])
X = X[featuers]
y_1 = pd.read_csv('y_data_1.csv', sep=',')
y_2 = pd.read_csv('y_data_2.csv', sep=',')
y = y_2.join(y_1)

z = y.join(X)
x2 = z[featuers2]
y2 = z[target2]


print("p6:", X.tail())
print("p7:", y.tail())
#TODO: create a second model, this one predicts the cog and heading, then after the prediction on the ais_test with both models, use it to update the shifted values to create a new prediction using the original model

X['vesselId'] = vessel_encoder.fit_transform(X['vesselId'])

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.01, random_state=rs)

X_train2, X_val2, y_train2, y_val2 = train_test_split(x2, y2, test_size=0.01, random_state=rs)

# Scale the features
scaler = StandardScaler()
scaler2 = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

X_train_scaled2 = scaler2.fit_transform(X_train2)
X_val_scaled2 = scaler2.transform(X_val2)


model = ExtraTreesRegressor(n_estimators=1, random_state=rs)
model2 = ExtraTreesRegressor(n_estimators=1, random_state=rs)
# Train a RandomForest model
model.fit(X_train_scaled, y_train)
model2.fit(X_train_scaled2, y_train2)

# Predict on validation set
y_pred = model.predict(X_val_scaled)

# Evaluate the model
mse = mean_squared_error(y_val, y_pred)
print(f'Mean Absolute Error: {mse}')
print(f'Root Mean Squared Error: {np.sqrt(mse)}')
print(f'R2-score: {r2_score(y_val, y_pred)}')
print(f'Variance Score: {explained_variance_score(y_val, y_pred)}')

# Use the same features as training
X_0 = pd.read_csv('x_test.csv', sep=',')
X_1 = pd.read_csv('x_test_1.csv', sep=',')
X_2 = pd.read_csv('x_test_2.csv', sep=',')
X_3 = pd.read_csv('x_test_3.csv', sep=',')
X_4 = pd.read_csv('x_test_4.csv', sep=',')
X_test = X_0.join([X_1, X_2, X_3, X_4])
X_test = X_test[featuers]
X_test['vesselId'] = vessel_encoder.transform(X_test['vesselId'])

X_test.to_excel("x_test.xlsx")
# Scale the test set
X_test_scaled = scaler.transform(X_test)

# Predict on the test set
test_predictions = model.predict(X_test_scaled)

# Save the predictions
predictions_df = pd.DataFrame(test_predictions, columns=['longitude', 'latitude'])
last = X_test.join(predictions_df)
X_test2 = X_test.join(predictions_df)

for i in range(1, timesteps):  # Only 3 previous time steps
    X_test2[f'lat_shift_{i}'] = X_test2.groupby('vesselId')['latitude'].shift(i)
    X_test2[f'lon_shift_{i}'] = X_test2.groupby('vesselId')['longitude'].shift(i)
    #X_test2[f'minute_shift_{i}'] = X_test2.groupby('vesselId')['minute'].shift(i)
    #X_test2[f'hour_shift_{i}'] = X_test2.groupby('vesselId')['hour'].shift(i)
    #X_test2[f'day_shift_{i}'] = X_test2.groupby('vesselId')['day'].shift(i)

    X_test2[f'lat_shift_{i}'] = np.where(X_test2[f'lat_shift_{i}'].isna(), last[f'lat_shift_{i}'], X_test2[f'lat_shift_{i}'])
    X_test2[f'lon_shift_{i}'] = np.where(X_test2[f'lon_shift_{i}'].isna(), last[f'lon_shift_{i}'], X_test2[f'lon_shift_{i}'])
    #X_test2[f'minute_shift_{i}'] = np.where(X_test2[f'minute_shift_{i}'].isna(), last[f'minute_shift_{i}'], X_test2[f'minute_shift_{i}'])
    #X_test2[f'hour_shift_{i}'] = np.where(X_test2[f'hour_shift_{i}'].isna(), last[f'hour_shift_{i}'], X_test2[f'hour_shift_{i}'])
    #X_test2[f'day_shift_{i}'] = np.where(X_test2[f'day_shift_{i}'].isna(), last[f'day_shift_{i}'], X_test2[f'day_shift_{i}'])

last = X_test2[featuers]
X_test2 = X_test2[featuers2]
X_test_scaled2 = scaler2.transform(X_test2)
#TODO: use predictions_df in the slides for the next two predicts


test_predictions2 = model2.predict(X_test_scaled2)
headercogup = pd.DataFrame(test_predictions2, columns=target2)

last['cog'] = headercogup['cog']
last['heading'] = headercogup['heading']

last.to_excel("last.xlsx")

last_scaled = scaler.transform(last)

final_prediction = model.predict(last_scaled)

predictions_df = pd.DataFrame(final_prediction, columns=['longitude_predicted', 'latitude_predicted'])
predictions_df.to_csv('ais_test_predictions_1.csv', index=True)
print(datetime.datetime.today().strftime("%HH:%MM %dd"))



20H:33M 24d
p6:          vesselId  hour  day  month  minute  vesselType  length  breadth   
1522060       682    23    7      5      59        83.0  182.00     25.0  \
1522061        85    23    7      5      59        83.0  199.00     32.0   
1522062       459    23    7      5      59        83.0  199.96     38.0   
1522063       596    23    7      5      59        21.0  186.00     27.0   
1522064       637    23    7      5      59        83.0  200.00     32.0   

          CEU      DWT  ...  lon_shift_4  lat_shift_5  lon_shift_5    cog   
1522060   300  12502.0  ...     -5.84482     51.09372     -5.84536  359.1  \
1522061  6354  22160.0  ...    -12.10865     38.47765    -12.13535   12.3   
1522062  7429  18241.0  ...     -4.66703     49.77723     -4.50058  269.8   
1522063  1400   7150.0  ...     10.29855     37.64190     11.32413    8.0   
1522064  6215  18907.0  ...    -74.95443     38.70218    -74.90653  336.0   

         heading  time_diff_1  time_diff_2  time_diff_3  time_di